In [10]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [11]:
data = np.random.randint(1, 100, size=(100000, 3))
df = pd.DataFrame(data, columns=['a', 'b', 'c'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   a       100000 non-null  int64
 1   b       100000 non-null  int64
 2   c       100000 non-null  int64
dtypes: int64(3)
memory usage: 2.3 MB


In [12]:
df.head()

,a,b,c
0,54,4,17
1,60,36,6
2,94,85,86
3,10,20,29
4,59,3,94


In [13]:
grouped = df.groupby(['b', 'c'], sort=False)['a'].agg(['median', 'mean', 'min', 'max', 'std', 'count'])
grouped.head()

,,median,mean,min,max,std,count
b,c,,,,,,
4,17,67.0,55.090909,9,88,26.655035,11
36,6,51.0,46.857143,11,98,35.960296,7
85,86,37.0,48.375000,4,94,36.943731,8
20,29,42.0,43.909091,9,98,27.285361,11
3,94,29.0,41.133333,1,97,31.368471,15


In [14]:
dict(grouped.loc[(1, 1)])

{'median': 70.0,
 'mean': 62.25,
 'min': 19.0,
 'max': 96.0,
 'std': 26.15035198789701,
 'count': 12.0}

In [15]:
quantiles = df.quantile([i / 10 for i in range(1, 10)], interpolation='linear')
quantiles.head(10)

,a,b,c
0.1,10.0,10.0,10.0
0.2,20.0,20.0,20.0
0.3,30.0,30.0,30.0
0.4,40.0,40.0,40.0
0.5,50.0,50.0,50.0
0.6,60.0,60.0,60.0
0.7,70.0,70.0,70.0
0.8,80.0,80.0,80.0
0.9,90.0,90.0,90.0


In [16]:
dict(quantiles.loc[0.2])

{'a': 20.0, 'b': 20.0, 'c': 20.0}

In [17]:
rows = []
for t in tqdm(df.itertuples()):
    row = t._asdict()
    g = grouped.loc[(row['b'], row['c'])]
    row['median'] = g['median']
    row['mean'] = g['mean']
    row['min'] = g['min']
    row['max'] = g['max']
    row['std'] = g['std']
    rows.append(row)

100000it [00:18, 5543.28it/s]


In [18]:
%%time
res = pd.DataFrame.from_records(rows)
# prevent division-by-zero error
eps = np.finfo(np.float32).eps
res['std'].fillna(eps, inplace=True)
res.set_index('Index', drop=True, inplace=True)
res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   a       100000 non-null  int64  
 1   b       100000 non-null  int64  
 2   c       100000 non-null  int64  
 3   median  100000 non-null  float64
 4   mean    100000 non-null  float64
 5   min     100000 non-null  float64
 6   max     100000 non-null  float64
 7   std     100000 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 6.9 MB
CPU times: user 250 ms, sys: 31.2 ms, total: 281 ms
Wall time: 274 ms


In [19]:
res.head()

,a,b,c,median,mean,min,max,std
Index,,,,,,,,
0,54,4,17,67.0,55.090909,9.0,88.0,26.655035
1,60,36,6,51.0,46.857143,11.0,98.0,35.960296
2,94,85,86,37.0,48.375000,4.0,94.0,36.943731
3,10,20,29,42.0,43.909091,9.0,98.0,27.285361
4,59,3,94,29.0,41.133333,1.0,97.0,31.368471
